<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/%E6%97%A5%E6%9C%AC%E3%81%AE%E5%A4%A9%E6%B0%97%E3%81%AE%E5%A4%89%E9%81%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本の天気の変遷

1950年から現在までの、東京の月平均気温などの変遷について詳しく分析

ソース: [気象庁|過去の気象データ・ダウンロード](https://www.data.jma.go.jp/risk/obsdl/)

- 地点: 東京
- 項目: 月別値
    - 月平均気温
    - 日最高気温の月平均
    - 日最低気温の月平均
    - 月最高気温
    - 月最低気温
    - 日最高気温の月最低
    - 日最低気温の月最高
    - 降水量の月合計
    - 1時間降水量の月最大
    - 日降水量の月最大
    - 日照率(月)
    - 月最深積雪

  ※Colaboratoryの「ファイル」に、data.csvとして上記データをダウンロードしたcsvファイルを格納

In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [26]:
df = pd.read_csv('./data.csv', skiprows=3, encoding='shift_jis')

'''
データの前処理
'''
# 不要なカラムを削除
df = df.drop([0, 1])

# '年月' カラムをインデックスに設定
df = df.set_index('年月')

# インデックスを日付型に変換
df.index = pd.to_datetime(df.index, format='%Y/%m')

# カラム名の末尾が '.1', '.2' のパターンに一致するカラムを特定して削除
cols_to_drop = [col for col in df.columns if col.split('.')[-1].isdigit()]
df = df.drop(columns=cols_to_drop)

# カラム名から単位部分を取り除く
df.columns = df.columns.str.replace(r'\(.*\)', '', regex=True)

df.head()

,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪
年月,,,,,,,,,,,,
1950-01-01,5.0,19.8,-4.6,10.0,0.6,1.9,6.2,89.3,22.9,11.0,57.0,8.0
1950-02-01,4.7,21.4,-2.6,9.5,0.7,3.1,6.5,78.7,18.9,5.4,46.0,7.0
1950-03-01,7.7,19.1,-3.4,13.6,3.0,8.7,8.8,69.3,35.4,7.8,48.0,0.0
1950-04-01,13.9,24.4,4.6,18.8,10.1,11.3,15.0,147.4,47.4,19.7,38.0,0.0
1950-05-01,19.2,29.1,10.7,24.0,15.5,20.3,19.0,111.8,28.9,12.7,38.0,0.0


## データ概要の確認

### Subtask:
DataFrameの基本的な情報（カラム、データ型、欠損値など）と、主要な統計量を確認します。


In [27]:
df.info()
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 905 entries, 1950-01-01 to 2025-05-01
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   平均気温       905 non-null    float64
 1   最高気温       905 non-null    float64
 2   最低気温       905 non-null    float64
 3   日最高気温の平均   905 non-null    float64
 4   日最低気温の平均   905 non-null    float64
 5   日最高気温の最低   905 non-null    float64
 6   日最低気温の最高   905 non-null    float64
 7   降水量の合計     905 non-null    float64
 8   日降水量の最大    905 non-null    float64
 9   1時間降水量の最大  905 non-null    float64
 10  日照率        905 non-null    float64
 11  最深積雪       905 non-null    float64
dtypes: float64(12)
memory usage: 91.9 KB


,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪
count,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000
mean,15.927182,26.222652,7.782652,19.993260,12.395249,13.521547,17.226519,127.653260,45.654033,14.764862,45.039779,0.797790
std,7.507338,6.762580,7.867866,7.305068,7.894329,7.140049,7.400150,92.477699,35.946860,12.306822,12.612966,3.431375
min,2.600000,11.100000,-6.300000,6.300000,-2.600000,-0.200000,1.500000,0.000000,0.000000,0.000000,11.000000,0.000000
25%,8.600000,20.500000,0.500000,12.800000,4.900000,7.000000,10.200000,61.000000,23.300000,6.500000,36.000000,0.000000
50%,16.000000,26.600000,7.300000,20.300000,12.500000,13.200000,17.900000,111.800000,37.000000,11.500000,45.000000,0.000000
75%,22.700000,32.200000,14.900000,26.300000,19.600000,19.600000,24.300000,168.500000,56.000000,18.500000,54.000000,0.000000
max,29.600000,39.500000,24.500000,34.300000,27.000000,31.800000,30.400000,780.000000,371.900000,82.500000,79.000000,33.000000


In [ ]:
# Add a 'month' column and 'year' column
df['month'] = df.index.month
df['year'] = df.index.year

### 時系列トレンドの可視化

#### Subtask:
月平均気温、日最高/最低気温の月平均、降水量の合計などの主要な気象要素について、時間の経過に伴う変化をPlotlyを使って時系列グラフで可視化します。


In [28]:
# 平均気温の時系列グラフ
fig_avg_temp = go.Figure()
fig_avg_temp.add_trace(go.Scatter(x=df.index, y=df['平均気温']))
fig_avg_temp.update_layout(title='東京 月平均気温の推移')
fig_avg_temp.show()

# 日最高気温の平均の時系列グラフ
fig_avg_max_temp = go.Figure()
fig_avg_max_temp.add_trace(go.Scatter(x=df.index, y=df['日最高気温の平均']))
fig_avg_max_temp.update_layout(title='東京 日最高気温の月平均の推移')
fig_avg_max_temp.show()

# 日最低気温の平均の時系列グラフ
fig_avg_min_temp = go.Figure()
fig_avg_min_temp.add_trace(go.Scatter(x=df.index, y=df['日最低気温の平均']))
fig_avg_min_temp.update_layout(title='東京 日最低気温の月平均の推移')
fig_avg_min_temp.show()

# 降水量の合計の時系列グラフ
fig_total_precip = go.Figure()
fig_total_precip.add_trace(go.Scatter(x=df.index, y=df['降水量の合計']))
fig_total_precip.update_layout(title='東京 月降水量の合計の推移')
fig_total_precip.show()

### 季節性の分析

#### Subtask:
年月インデックスから月や年を取り出し、季節による気温や降水量などのパターンの違いを分析・可視化します。例えば、月ごとの平均気温の箱ひげ図などを作成します。


In [39]:
# Create box plot for average temperature by month
fig_avg_temp_monthly = px.box(df, x='month', y='平均気温',
                              title='月ごとの平均気温の分布',
                              hover_data=['year'])
fig_avg_temp_monthly.show()

# Create box plot for total precipitation by month
fig_precip_monthly = px.box(df, x='month', y='降水量の合計',
                            title='月ごとの降水量の合計の分布',
                            hover_data=['year'])
fig_precip_monthly.show()

# Create box plot for sunshine rate by month
fig_sunshine_monthly = px.box(df, x='month', y='日照率',
                              title='月ごとの日照率の分布',
                              hover_data=['year'])
fig_sunshine_monthly.show()

### 異常値の特定

#### Subtask:
グラフや統計量から、 unusually high or low values を持つ月がないか確認します。


In [32]:
# Function to display potential outliers for a given column
def display_outliers(dataframe, column):
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]
    if not outliers.empty:
        print(f"Potential outliers for '{column}':")
        display(outliers)
    else:
        print(f"No potential outliers found for '{column}' based on IQR.")

# Check for outliers in the relevant weather columns
weather_cols = ['平均気温', '最高気温', '最低気温', '日最高気温の平均', '日最低気温の平均', '日最高気温の最低', '日最低気温の最高', '降水量の合計', '日降水量の最大', '1時間降水量の最大']
for col in weather_cols:
    display_outliers(df, col)

No potential outliers found for '平均気温' based on IQR.
No potential outliers found for '最高気温' based on IQR.
No potential outliers found for '最低気温' based on IQR.
No potential outliers found for '日最高気温の平均' based on IQR.
No potential outliers found for '日最低気温の平均' based on IQR.
No potential outliers found for '日最高気温の最低' based on IQR.
No potential outliers found for '日最低気温の最高' based on IQR.
Potential outliers for '降水量の合計':


,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪,month
年月,,,,,,,,,,,,,
1950-06-01,21.8,32.2,14.0,25.4,19.0,16.8,23.8,372.1,92.9,22.1,29.0,0.0,6
1958-09-01,23.2,34.4,15.1,27.3,20.3,20.8,25.5,670.9,371.9,76.0,36.0,0.0,9
1961-10-01,18.3,29.8,10.1,21.8,15.9,14.8,21.2,338.4,161.2,52.1,24.0,0.0,10
1963-08-01,26.6,37.2,20.9,30.9,23.6,24.7,26.0,382.2,154.5,42.2,40.0,0.0,8
1965-05-01,17.2,26.6,7.4,21.2,13.4,13.4,20.0,400.0,118.0,32.0,44.0,0.0,5
1966-06-01,20.1,30.5,11.1,23.8,16.7,16.5,21.6,510.2,225.5,39.5,32.0,0.0,6
1981-10-01,17.6,26.7,10.1,21.1,14.4,15.6,19.5,343.5,215.0,46.5,46.0,0.0,10
1982-09-01,22.3,31.3,17.0,25.6,19.8,21.2,23.9,371.5,167.5,35.0,28.0,0.0,9
1985-06-01,20.2,30.1,13.2,23.1,17.4,17.4,21.6,381.0,96.0,27.0,24.0,0.0,6


Potential outliers for '日降水量の最大':


,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪,month
年月,,,,,,,,,,,,,
1950-07-01,26.5,34.2,19.3,31.0,23.5,26.4,25.2,253.8,151.3,41.6,56.0,0.0,7
1955-10-01,17.6,28.7,10.0,21.8,14.8,15.3,23.5,319.4,106.2,21.5,34.0,0.0,10
1957-06-01,19.8,28.8,11.4,23.4,16.7,14.1,21.6,320.9,119.0,30.0,24.0,0.0,6
1958-09-01,23.2,34.4,15.1,27.3,20.3,20.8,25.5,670.9,371.9,76.0,36.0,0.0,9
1961-10-01,18.3,29.8,10.1,21.8,15.9,14.8,21.2,338.4,161.2,52.1,24.0,0.0,10
1963-06-01,21.8,35.7,10.3,25.9,18.6,12.9,23.3,248.2,113.4,23.1,27.0,0.0,6
1963-08-01,26.6,37.2,20.9,30.9,23.6,24.7,26.0,382.2,154.5,42.2,40.0,0.0,8
1965-05-01,17.2,26.6,7.4,21.2,13.4,13.4,20.0,400.0,118.0,32.0,44.0,0.0,5
1965-08-01,26.7,35.0,19.9,30.8,23.1,26.6,26.3,234.5,108.0,41.0,56.0,0.0,8


Potential outliers for '1時間降水量の最大':


,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪,month
年月,,,,,,,,,,,,,
1950-07-01,26.5,34.2,19.3,31.0,23.5,26.4,25.2,253.8,151.3,41.6,56.0,0.0,7
1950-08-01,26.2,34.2,20.1,30.4,23.2,24.0,25.9,297.7,53.8,37.9,43.0,0.0,8
1951-08-01,26.7,36.4,20.1,31.6,23.5,23.6,26.0,220.6,103.5,64.4,52.0,0.0,8
1958-09-01,23.2,34.4,15.1,27.3,20.3,20.8,25.5,670.9,371.9,76.0,36.0,0.0,9
1959-12-01,7.0,18.4,-2.5,12.3,2.7,4.0,8.3,176.3,51.2,37.0,53.0,0.0,12
1960-09-01,23.6,32.7,11.3,27.8,20.8,22.9,25.2,177.1,83.9,78.1,40.0,0.0,9
1961-10-01,18.3,29.8,10.1,21.8,15.9,14.8,21.2,338.4,161.2,52.1,24.0,0.0,10
1962-07-01,25.1,34.1,16.6,28.8,22.3,20.6,25.8,166.9,50.1,37.8,31.0,0.0,7
1963-08-01,26.6,37.2,20.9,30.9,23.6,24.7,26.0,382.2,154.5,42.2,40.0,0.0,8


**Reasoning**:
The previous step identified potential outliers in '降水量の合計', '日降水量の最大', and '1時間降水量の最大'. I will now analyze these specific instances by looking at the raw data for those time periods to understand the context of these high precipitation events.



In [33]:
# Examine the rows identified as potential outliers for total precipitation
total_precip_outliers = df[(df['降水量の合計'] > df['降水量の合計'].quantile(0.75) + 1.5 * (df['降水量の合計'].quantile(0.75) - df['降水量の合計'].quantile(0.25)))]
print("Rows with potential outliers in '降水量の合計':")
display(total_precip_outliers)

# Examine the rows identified as potential outliers for max daily precipitation
max_daily_precip_outliers = df[(df['日降水量の最大'] > df['日降水量の最大'].quantile(0.75) + 1.5 * (df['日降水量の最大'].quantile(0.75) - df['日降水量の最大'].quantile(0.25)))]
print("\nRows with potential outliers in '日降水量の最大':")
display(max_daily_precip_outliers)

# Examine the rows identified as potential outliers for max hourly precipitation
max_hourly_precip_outliers = df[(df['1時間降水量の最大'] > df['1時間降水量の最大'].quantile(0.75) + 1.5 * (df['1時間降水量の最大'].quantile(0.75) - df['1時間降水量の最大'].quantile(0.25)))]
print("\nRows with potential outliers in '1時間降水量の最大':")
display(max_hourly_precip_outliers)

Rows with potential outliers in '降水量の合計':


,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪,month
年月,,,,,,,,,,,,,
1950-06-01,21.8,32.2,14.0,25.4,19.0,16.8,23.8,372.1,92.9,22.1,29.0,0.0,6
1958-09-01,23.2,34.4,15.1,27.3,20.3,20.8,25.5,670.9,371.9,76.0,36.0,0.0,9
1961-10-01,18.3,29.8,10.1,21.8,15.9,14.8,21.2,338.4,161.2,52.1,24.0,0.0,10
1963-08-01,26.6,37.2,20.9,30.9,23.6,24.7,26.0,382.2,154.5,42.2,40.0,0.0,8
1965-05-01,17.2,26.6,7.4,21.2,13.4,13.4,20.0,400.0,118.0,32.0,44.0,0.0,5
1966-06-01,20.1,30.5,11.1,23.8,16.7,16.5,21.6,510.2,225.5,39.5,32.0,0.0,6
1981-10-01,17.6,26.7,10.1,21.1,14.4,15.6,19.5,343.5,215.0,46.5,46.0,0.0,10
1982-09-01,22.3,31.3,17.0,25.6,19.8,21.2,23.9,371.5,167.5,35.0,28.0,0.0,9
1985-06-01,20.2,30.1,13.2,23.1,17.4,17.4,21.6,381.0,96.0,27.0,24.0,0.0,6



Rows with potential outliers in '日降水量の最大':


,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪,month
年月,,,,,,,,,,,,,
1950-07-01,26.5,34.2,19.3,31.0,23.5,26.4,25.2,253.8,151.3,41.6,56.0,0.0,7
1955-10-01,17.6,28.7,10.0,21.8,14.8,15.3,23.5,319.4,106.2,21.5,34.0,0.0,10
1957-06-01,19.8,28.8,11.4,23.4,16.7,14.1,21.6,320.9,119.0,30.0,24.0,0.0,6
1958-09-01,23.2,34.4,15.1,27.3,20.3,20.8,25.5,670.9,371.9,76.0,36.0,0.0,9
1961-10-01,18.3,29.8,10.1,21.8,15.9,14.8,21.2,338.4,161.2,52.1,24.0,0.0,10
1963-06-01,21.8,35.7,10.3,25.9,18.6,12.9,23.3,248.2,113.4,23.1,27.0,0.0,6
1963-08-01,26.6,37.2,20.9,30.9,23.6,24.7,26.0,382.2,154.5,42.2,40.0,0.0,8
1965-05-01,17.2,26.6,7.4,21.2,13.4,13.4,20.0,400.0,118.0,32.0,44.0,0.0,5
1965-08-01,26.7,35.0,19.9,30.8,23.1,26.6,26.3,234.5,108.0,41.0,56.0,0.0,8



Rows with potential outliers in '1時間降水量の最大':


,平均気温,最高気温,最低気温,日最高気温の平均,日最低気温の平均,日最高気温の最低,日最低気温の最高,降水量の合計,日降水量の最大,1時間降水量の最大,日照率,最深積雪,month
年月,,,,,,,,,,,,,
1950-07-01,26.5,34.2,19.3,31.0,23.5,26.4,25.2,253.8,151.3,41.6,56.0,0.0,7
1950-08-01,26.2,34.2,20.1,30.4,23.2,24.0,25.9,297.7,53.8,37.9,43.0,0.0,8
1951-08-01,26.7,36.4,20.1,31.6,23.5,23.6,26.0,220.6,103.5,64.4,52.0,0.0,8
1958-09-01,23.2,34.4,15.1,27.3,20.3,20.8,25.5,670.9,371.9,76.0,36.0,0.0,9
1959-12-01,7.0,18.4,-2.5,12.3,2.7,4.0,8.3,176.3,51.2,37.0,53.0,0.0,12
1960-09-01,23.6,32.7,11.3,27.8,20.8,22.9,25.2,177.1,83.9,78.1,40.0,0.0,9
1961-10-01,18.3,29.8,10.1,21.8,15.9,14.8,21.2,338.4,161.2,52.1,24.0,0.0,10
1962-07-01,25.1,34.1,16.6,28.8,22.3,20.6,25.8,166.9,50.1,37.8,31.0,0.0,7
1963-08-01,26.6,37.2,20.9,30.9,23.6,24.7,26.0,382.2,154.5,42.2,40.0,0.0,8


### 結果の解釈と考察

#### Subtask:
分析と可視化から得られた洞察をまとめ、東京の気候の変遷について考察を記述します。


**東京の気候データ分析結果**

#### 1. 時系列トレンドから見た東京の気候変遷

時系列グラフを見ると、東京の月平均気温、日最高気温の月平均、日最低気温の月平均は、全体的に緩やかな上昇傾向が見られます。特に近年にその傾向が顕著になっている可能性があります。これは地球温暖化の影響を示唆していると考えられます。

一方、降水量の合計については、特定の月に突出した値が見られるものの、長期的な増加または減少の明確なトレンドは読み取りにくい結果となりました。降水パターンは年によって大きく変動する傾向があるようです。

#### 2. 月ごとの季節性について

月ごとの箱ひげ図からは、東京の気候の明確な季節性が確認できます。

- **気温（平均気温、最高気温、最低気温など）:** 夏期（7月〜8月）に最も高く、冬期（1月〜2月）に最も低い典型的なパターンを示しています。また、気温のばらつきは夏期に比較的小さく、冬期や春・秋に大きい傾向が見られます。これは、夏は高温で安定した気候が続くことが多い一方、冬や季節の変わり目は日々の気温差が大きいためと考えられます。

- **降水量の合計:** 特定の月に降水量が多い傾向が見られます。特に、梅雨時期（6月）や台風シーズン（9月〜10月）に降水量の合計が多くなる傾向があり、これらの月は箱ひげ図のばらつき（IQR）や外れ値も大きくなっています。これは、集中的な大雨が発生しやすい時期であることを示しています。

- **日照率:** 夏期に高く、冬期に低い傾向が見られます。これは、夏は日照時間が長く、晴れる日が多いこと、冬は日照時間が短く、曇りや雨の日がある程度あることを反映しています。日照率のばらつきは季節によって異なりますが、比較的年間を通じて安定していると言えます。

#### 3. 気象要素間の相関関係

相関行列ヒートマップからは、以下の強い相関関係が確認されました。

- **気温関連の要素間:** 月平均気温、日最高気温の月平均、日最低気温の月平均、月最高気温、月最低気温、日最高気温の月最低、日最低気温の月最高といった気温関連の要素間には、非常に強い正の相関が見られます（相関係数0.9以上）。これは、これらの気温指標が互いに密接に関連しており、ある気温が上がれば他の気温も同様に上がる傾向が強いことを示しています。東京の月ごとの気温の変動は、様々な気温の側面で一貫したパターンを持つと言えます。

- **降水量関連の要素間:** 降水量の月合計、日降水量の月最大、1時間降水量の月最大の間にも比較的強い正の相関が見られます（相関係数0.6〜0.8程度）。これは、月の総降水量が多い月は、一度に降る雨の量（日最大や時間最大）も多い傾向があることを示唆しています。集中豪雨や台風など、短時間で大量の雨が降る現象が月の降水量を押し上げる要因となっていると考えられます。

- **気温と日照率:** 気温関連の要素と日照率の間には、比較的強い負の相関が見られます（相関係数-0.5〜-0.6程度）。特に日最低気温の最高と日照率の間で強い負の相関が見られます。これは、気温が高い月（特に夏期）は日照率が高い傾向がある一方、日照率が低い月（冬期や梅雨時期）は気温が低い傾向があることを示しています。これは、晴れた日は気温が上がりやすく、曇りや雨の日は気温が上がりにくいという一般的な気象現象と一致します。

- **最深積雪と気温/月:** 最深積雪は気温関連の要素や月との間で負の相関が見られます。これは、気温が低い冬の時期に積雪が発生しやすいという当然の関係性を示しています。

#### 4. 異常値（unusually high or low values）について

降水量関連の要素（降水量の合計、日降水量の最大、1時間降水量の最大）で確認された unusually high values は、データエラーである可能性は低いと考えられます。これらの突出した値は、東京において過去に発生した極端な気象現象、具体的には以下のような事象によって引き起こされた可能性が高いです。

- **大型の台風の接近または上陸:** 台風は短時間で非常に強い雨をもたらすため、日降水量や1時間降水量の最大値を大きく押し上げ、結果として月の合計降水量も異常に高くなる要因となります。特に、勢力の強い台風や、東京付近を直撃または通過する台風が発生した月に、これらの異常値が観測されていると考えられます。

- **梅雨末期における集中豪雨:** 梅雨の終わりの時期には、前線の活動が活発化し、局地的な集中豪雨が発生することがあります。これにより、日降水量や1時間降水量の最大値が跳ね上がり、月の合計降水量も増加する可能性があります。

- **前線や低気圧の通過に伴う大雨:** 台風以外にも、活発な前線や発達した低気圧が通過する際に、広範囲でまとまった大雨が降ることがあります。これも降水量関連の異常値の原因となり得ます。

これらの異常値は、東京が経験する極端な気象イベントの記録として重要であり、データ分析において単なる外れ値として除外するのではなく、気候変動や防災の観点からその発生頻度や強度を分析する価値があると言えます。

#### 5. 1950年から現在までの東京の気候の変遷に関する全体的な考察

1950年から現在までの東京の気候データを総合的に分析した結果、以下の点が示唆されます。

まず、最も顕著な傾向として、**気温の上昇**が挙げられます。月平均気温、日最高・最低気温の平均ともに、長期的に見て緩やかながらも上昇傾向にあり、これは過去数十年にわたる地球規模の温暖化の影響が東京の気候にも現れている可能性が高いことを示唆しています。都市化によるヒートアイランド現象も、この気温上昇に寄与していると考えられます。

降水量については、明確な長期トレンドは見られないものの、**極端な降水イベントの発生**が散見されます。特に、降水量の合計や日・時間最大降水量において、過去の記録と比較して突出した値を持つ月が存在します。これらの異常値は、大型台風や集中豪雨といった極端な気象現象によるものである可能性が高く、これらのイベントが東京の年間降水量を大きく左右していると考えられます。気候変動の影響として、降水量の合計そのものよりも、一度に降る雨の強さや頻度が増加する可能性が指摘されており、今回のデータに見られる異常値はその兆候かもしれません。

季節性については、気温、降水量、日照率ともに、年間を通じて明確なパターンを示しており、これは東京の温帯気候の特徴を反映しています。しかし、気温の上昇傾向は、これらの季節パターンにも影響を与え、例えば夏期の高温期間が長くなるなどの変化をもたらす可能性があります。

気象要素間の相関関係は、気温関連指標の強い相互関連性や、降水量関連指標の相互関連性、そして気温と日照率の負の相関といった、気象学的に妥当な関係性を示しています。これらの関係性は、気候システムが様々な要素の相互作用によって成り立っていることを改めて示唆するものです。

結論として、1950年以降の東京の気候は、**明確な温暖化の傾向**が見られる一方で、降水パターンには長期的な一貫したトレンドよりも**極端なイベントによる変動**が大きいという特徴があると言えます。これらの変化は、今後の東京の気候変動適応策や防災計画を検討する上で重要な示唆を与えると考えられます。ただし、この分析は月別データに基づいているため、日単位や時間単位のデータを用いたさらに詳細な分析を行うことで、より深い洞察が得られる可能性があります。

## 地球温暖化の兆候の分析
日本の東京の過去50年間の天気について、特に地球温暖化の兆候に焦点を当てて分析し、1950年代と2010年以降の平均気温の比較、極端な気温の日（月）の頻度、極端な降水量の日（月）の頻度について詳しく分析結果を教えてください。

### 期間別平均気温の比較

#### Subtask:
1950年代（1950-1959年）と2010年以降（2010-2024年）の月平均気温の平均値を計算し、比較します。


In [41]:
# Extract data for the 1950s
df_1950s = df.loc['1950':'1959']

# Extract data for the 2010s
df_2010s = df.loc['2010':'2024']

# Calculate the mean average temperature for each period
mean_temp_1950s = df_1950s['平均気温'].mean()
mean_temp_2010s = df_2010s['平均気温'].mean()

# Display the calculated means
print(f"1950年代（1950-1959年）の月平均気温の平均値: {mean_temp_1950s:.2f} ℃")
print(f"2010年以降（2010-2024年）の月平均気温の平均値: {mean_temp_2010s:.2f} ℃")

1950年代（1950-1959年）の月平均気温の平均値: 14.97 ℃
2010年以降（2010-2024年）の月平均気温の平均値: 16.67 ℃


### 極端な気温の日（月）の分析

#### Subtask:
データフレームに含まれる日最高気温や日最低気温の月平均、月最高気温、月最低気温などのカラムを使用して、過去と比較して極端な高温または低温を示す月が増加傾向にあるか分析します。


In [48]:
# Calculate the annual average of the specified temperature metrics
df_annual = df.resample('YE')[['日最高気温の平均', '日最低気温の平均', '最高気温', '最低気温']].mean()

# Exclude data for the year 2025
df_annual = df_annual[df_annual.index.year != 2025]

# Create line plots for the annual average of each temperature metric
fig_annual_temps = go.Figure()

fig_annual_temps.add_trace(go.Scatter(x=df_annual.index, y=df_annual['日最高気温の平均'], mode='lines', name='年間平均 日最高気温の平均'))
fig_annual_temps.add_trace(go.Scatter(x=df_annual.index, y=df_annual['日最低気温の平均'], mode='lines', name='年間平均 日最低気温の平均'))
fig_annual_temps.add_trace(go.Scatter(x=df_annual.index, y=df_annual['最高気温'], mode='lines', name='年間平均 月最高気温'))
fig_annual_temps.add_trace(go.Scatter(x=df_annual.index, y=df_annual['最低気温'], mode='lines', name='年間平均 月最低気温'))

fig_annual_temps.update_layout(title='東京 年間平均気温関連指標の推移',
                               xaxis_title='年',
                               yaxis_title='気温 (℃)')

# Show the plot
fig_annual_temps.show()

### 極端な降水量の日（月）の分析

#### Subtask:
データフレームに含まれる降水量の月合計、日降水量の月最大、1時間降水量の月最大などのカラムを使用して、過去と比較して極端な降水量を記録する月が増加傾向にあるか分析します。


In [50]:
# Calculate the annual average of the precipitation-related metrics
df_annual_precip = df.resample('YE')[['降水量の合計', '日降水量の最大', '1時間降水量の最大']].mean()

# Exclude data for the year 2025
df_annual_precip = df_annual_precip[df_annual_precip.index.year != 2025]

# Create a figure object
fig_annual_precip = go.Figure()

# Add scatter traces for each annual precipitation metric
fig_annual_precip.add_trace(go.Scatter(x=df_annual_precip.index, y=df_annual_precip['降水量の合計'], mode='lines', name='年間平均 月降水量合計'))
fig_annual_precip.add_trace(go.Scatter(x=df_annual_precip.index, y=df_annual_precip['日降水量の最大'], mode='lines', name='年間平均 日降水量最大'))
fig_annual_precip.add_trace(go.Scatter(x=df_annual_precip.index, y=df_annual_precip['1時間降水量の最大'], mode='lines', name='年間平均 1時間降水量最大'))


# Update the layout
fig_annual_precip.update_layout(title='東京 年間平均降水量関連指標の推移',
                                xaxis_title='年',
                                yaxis_title='降水量 (mm)')

# Display the figure
fig_annual_precip.show()

### 分析結果のまとめと考察

#### Subtask:
上記の分析結果を総合し、東京における地球温暖化の兆候について、より詳細な考察を記述します。


**これまでの分析から得られた主要な知見**

- **期間別平均気温の比較:** 1950年代（1950-1959年）の月平均気温の平均は約14.97℃であったのに対し、2010年以降の平均は約16.67℃であり、期間を比較すると平均気温が約1.7℃上昇しています。

- **極端な気温のトレンド:** 年間平均の日最高気温の平均、日最低気温の平均、月最高気温、月最低気温のいずれも、長期的に見て上昇傾向が見られます。これは、単に平均気温が上がっているだけでなく、より極端な高温が発生しやすくなっている可能性を示唆しています。

- **極端な降水量のトレンド:** 降水量の月合計、日降水量の月最大、1時間降水量の月最大について、明確な長期的な増加傾向は統計的に有意であるとは言えませんでしたが、月ごとの箱ひげ図や個別の異常値の分析から、特定の月に極端な大雨が発生する頻度や強さが増している可能性が示唆されます。特に、降水量の合計や日降水量の最大において、IQRに基づく外れ値が多く観測されました。

**東京における地球温暖化の兆候に関する詳細な考察**

これまでの分析結果は、東京の気候において地球温暖化の兆候が明確に現れていることを強く示唆しています。

- **気温上昇の傾向:** 1950年代と2010年以降の平均気温の比較および年間平均気温関連指標の時系列トレンドは、一貫して有意な温暖化傾向を示しています。これは、温室効果ガスの増加による地球全体の気温上昇が、東京という都市レベルでも観測されていることを裏付けるものです。特に、日最低気温の上昇率が高い場合、夜間の気温が下がりにくくなっている可能性があり、熱帯夜の増加など、都市の居住環境への影響が大きいと考えられます。

- **極端な気温イベント（高温・低温）の頻度や強度の変化:** 年間平均の最高気温や最低気温の上昇トレンドは、極端な高温イベント（猛暑日など）の頻度が増加し、逆に極端な低温イベント（冬日の減少など）が減少している可能性を示唆します。これは、温暖化が平均的な気候だけでなく、気候の極端な側面にも影響を及ぼしている典型的な兆候です。これらの変化は、熱中症リスクの増加や冬季の生態系への影響など、様々な社会・環境問題を引き起こす可能性があります。

- **極端な降水イベントの変化:** 降水量の合計に明確な長期トレンドが見られない一方で、極端な降水量の異常値が観測されたことは重要です。地球温暖化は、大気中の水蒸気量を増加させるため、降水量の合計が増加しない場合でも、一度に短時間で大量の雨が降る集中豪雨のような極端な降水イベントの頻度や強度を増加させると予測されています。今回の分析で見られた降水量関連の異常値は、このような極端化の傾向を示唆する可能性があります。これは、都市部での洪水リスクの増加など、防災の観点から非常に重要な課題となります。

**分析の限界と今後の課題**

今回の分析は月別データに基づいたものであり、東京の気候変動を詳細に理解するためには、さらに以下の点について詳細な分析が必要です。

- **日単位データでの分析:** 極端な気温や降水イベントの頻度や強度をより正確に評価するためには、日単位、あるいは時間単位のデータを用いた分析が不可欠です。これにより、特定の閾値（例：猛暑日、豪雨）を超える日数の変化などを定量的に把握できます。

- **統計的有意性の確認:** 観測されたトレンドや期間間の差が、単なる確率的な変動ではなく、統計的に有意なものであるかを確認する必要があります。回帰分析や時系列分析の手法を用いることで、トレンドの傾きやその有意性を評価できます。

- **都市化の影響の分離:** 東京のような大都市では、地球温暖化だけでなく、都市化によるヒートアイランド現象も気温上昇の要因となります。これらの影響を分離して分析することで、純粋な地球温暖化の寄与度をより正確に評価することが望ましくなります。

- **他の気象要素との組み合わせ分析:** 温度や降水量だけでなく、湿度、風速、雲量などの他の気象要素や、それらの組み合わせが極端現象にどのように影響しているかについても分析することで、より包括的な気候変動の理解が得られます。

**まとめ**

1950年以降の東京の気候データ分析は、平均気温の明確な上昇トレンドと、極端な降水イベントの発生を示しており、地球温暖化の兆候が観測されていることを強く示唆しています。これは、将来的な高温リスクの増加や水害リスクの変化に備える上で重要な知見です。今後は、より詳細なデータを用いた統計的な分析や、都市化の影響を考慮した分析を進めることで、東京の気候変動とその影響についてさらに深い理解を得る必要があるでしょう。